In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 100000)
import warnings
warnings.filterwarnings("ignore")

In [3]:
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
from sklearn.neighbors import kneighbors_graph

In [4]:
# Import processed dataframe

%store -r df_final_v_2
print(df_final_v_2.shape)
df_final_v_2.head()

(1421, 88)


,f_african,f_brazilian,f_colombian,f_connecticut,f_costa_rican,f_dominican,f_ecuadorian,f_honduran,f_indonesian,f_italian,f_jamaican,f_kentucky_fire_cured,f_mexican,f_nicaraguan,f_panamanian,f_pennsylvania,f_peruvian,f_sungrown,f_united_states,f_varied,w_2000,w_98,w_brazilian,w_cameroon,w_candela,w_connecticut,w_connecticut_broadleaf,w_corojo,w_costa_rican,w_criollo,w_dominican,w_ecuador,w_habana,w_habano,w_honduran,w_htl,w_indonesian,w_jalapa,w_kentucky_fire_cured,w_ligero,w_maduro,w_nicaraguan,w_oscuro,w_pennsylvania_broadleaf,w_rosado,w_san_andres,w_sumatra,w_sungrown,w_varied,pr__Full,pr__Medium,pr__Medium-Full,pr__Mellow,pr__Mellow-Medium,Fl__False,Fl__No,Fl__True,Fl__Varies,Fl__nan,Tip__No,Tip__Varies,Tip__Yes,O__Bahamas,O__Costa Rica,O__Dominican Republic,"O__Dominican Republic, Honduras","O__Dominican Republic, Nicaragua","O__Ecuador, Europe",O__Europe,"O__Europe, Honduras",O__Honduras,"O__Honduras, Nicaragua",O__Indonesia,O__Ireland,O__Italy,O__Nicaragua,"O__Nicaragua, United States",O__Philippines,O__Sri Lanka,O__United States,Pr__No,Pr__Varies,Pr__Yes,Sw__No,Sw__Varies,Sw__Yes,rating,rating_count
prod_name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
@ZUCAR,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0.000,0.000000
1876 Reserve,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0.856,0.263059
1876 Reserve Maduro,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0.848,0.136000
3x3 Tubos by Davidoff,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0.906,0.017882
5 Vegas Big Five,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0.872,0.040941


In [5]:
# Drop columns that are represented by columns with the opposite value

df_final_v_3 = df_final_v_2.drop(['Fl__False','Fl__No', 'Tip__No','Pr__No', 'Sw__No'],1)

In [6]:
df_final_v_3.shape

(1421, 83)

In [7]:
# Remove duplicates

df_final_v_4 = df_final_v_3[~df_final_v_3.index.duplicated(keep='first')]
df_final_v_4.shape

(1419, 83)

In [8]:
#df_final_v_4.to_pickle('../data/working_data/df_final_v_4.pkl')

In [9]:
print(df_final_v_4.shape)
print(df_final_v_2.shape)

(1419, 83)
(1421, 88)


In [10]:
df_final_v_4.head()

,f_african,f_brazilian,f_colombian,f_connecticut,f_costa_rican,f_dominican,f_ecuadorian,f_honduran,f_indonesian,f_italian,f_jamaican,f_kentucky_fire_cured,f_mexican,f_nicaraguan,f_panamanian,f_pennsylvania,f_peruvian,f_sungrown,f_united_states,f_varied,w_2000,w_98,w_brazilian,w_cameroon,w_candela,w_connecticut,w_connecticut_broadleaf,w_corojo,w_costa_rican,w_criollo,w_dominican,w_ecuador,w_habana,w_habano,w_honduran,w_htl,w_indonesian,w_jalapa,w_kentucky_fire_cured,w_ligero,w_maduro,w_nicaraguan,w_oscuro,w_pennsylvania_broadleaf,w_rosado,w_san_andres,w_sumatra,w_sungrown,w_varied,pr__Full,pr__Medium,pr__Medium-Full,pr__Mellow,pr__Mellow-Medium,Fl__True,Fl__Varies,Fl__nan,Tip__Varies,Tip__Yes,O__Bahamas,O__Costa Rica,O__Dominican Republic,"O__Dominican Republic, Honduras","O__Dominican Republic, Nicaragua","O__Ecuador, Europe",O__Europe,"O__Europe, Honduras",O__Honduras,"O__Honduras, Nicaragua",O__Indonesia,O__Ireland,O__Italy,O__Nicaragua,"O__Nicaragua, United States",O__Philippines,O__Sri Lanka,O__United States,Pr__Varies,Pr__Yes,Sw__Varies,Sw__Yes,rating,rating_count
prod_name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
@ZUCAR,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.000,0.000000
1876 Reserve,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.856,0.263059
1876 Reserve Maduro,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.848,0.136000
3x3 Tubos by Davidoff,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.906,0.017882
5 Vegas Big Five,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.872,0.040941


In [11]:
df_main = pd.read_pickle('../data/working_data/df_main.pkl')

In [12]:
df_main.head()

,prod_name,rating,rating_count,desc,profile,vitolas,Brand:,Flavored,Has Tip,Origin,Pressed,Profile,Shapes,Sweet,Wrapper,Binder,Filler
0,@ZUCAR,None,None,"Vanilla and sugar... yes please!Espinosa Cigars have always prided themselves on creating cigars of superior quality that offer their fans the right cigar for every occasion, moment, or mood they may find themselves in. Fan favorites like Espinosa Crema, that many have called the perfect morning cigar, and the Laranja Reserva, a more bold flavor cigar perfect for pairing with a nightcap, are just a few of their offerings. Now, Espinosa is reaching out to expand their fan base. Their sites set on the infused cigars lovers, with their newest release aimed at giving them the chance at finding their perfect cigar with in their portfilio. Enter @ZUCAR, Espinosa's first vanilla infused, sweet-tipped cigar. Originally debuted back in 2016, with no information other than a vanilla infusion and production out of the Dominican Republic, this blend has been fully remade by the company. The new version, while still produced in the Dominican Republic, now utilizes Dominican long-filler Tobacco and an infusion of vanilla and spice to help balance the sweetness provided by the sugar sweet-tip. Sporting the same phenomenal quality in construction and components, this cigar falls right in line with all of Espinosa's previous offerings, providing an exceptional herfing experience for whatever event life may throw your way. Who knows...you may have just found your new favorite cigar!",Mellow,"[Corona, Panatela, Robusto, Toro]",Espinosa,Varies,No,Dominican Republic,False,NaN,"Corona, Panatela, Robusto, Toro",True,Ecuador Sumatra,NaN,NaN
1,1876 Reserve,4.28,559,"Even gramps would be proud.On a trip to factories in the Dominican Republic some time ago, I was offered a boatload of a really nice, smooth handmade called 1876 Reserve. I’m guessing the manufacturer either drank too much Dominican rum or his abacus was busted. Don’t get me wrong, I certainly ain’t complaining because I think under normal circumstances it would’ve cost me twice as much to buy what I bought (I better remove his name from the mailing list so he doesn’t see this later). What I do know is that this is an incredibly mellow Dominican cigar that’s well worth our ridiculous 2-fer price. In fact, the buy-in here starts under two bucks apiece! Is that too cheap? Maybe it’s me that’s getting soft in the head. Oh well, if you like easy going cigars, you’ll love this deal.",Mellow,"[Churchill, Robusto, Toro, Torpedo]",NaN,False,No,Dominican Republic,False,NaN,"Churchill, Robusto, Toro, Torpedo",False,Connecticut,Sumatra,Dominican
2,1876 Reserve Maduro,4.24,289,"A dark horse from 1876 Reserve.Maestro....drumroll please. And now, without further delay, enter 1876 Reserve Maduro. That's right, the fan-favorite 1876 Reserve now comes draped in a dark, juicy maduro wrapper leaf. The profile is still relatively tame, yet the darker, richer leaf adds a welcomed layer of complexity. And better yet, the price is exactly the same, right around two bucks apiece. I don't like taking credit when it's not due, but I definitely deserve a pat on the back for this one. Well-made Dominican lovelies stuffed with aged Dominican long-fillers and sporting a toothy maduro wrapper, all for pennies on the dollar. You can put this one at the top of the list for best damn value around.",Mellow-Medium,"[Churchill, Robusto, Toro, Torpedo]",NaN,False,No,Dominican Republic,False,NaN,"Churchill, Robusto, Toro, Torpedo",False,Maduro,NaN,Dominican
3,3x3 Tubos by Davidoff,4.53,38,"3x3 = an Unbridled Winner! There’s tons of cigars that have gone the way of the Dodo bird over the last few years… but perhaps the most sought-after blend post-disappearing act is the famed 3x3. On paper, this cigar shouldn’t work. It’s just a collection of 9 cigars, packaged in three rows of three, that each come in aluminum tubes. But the secret to this one is in the sauce! Handmade by the

In [13]:
# Isolate product strength into a dataframe

df_strength = df_final_v_4[['pr__Full', 'pr__Medium-Full', 'pr__Medium', 'pr__Mellow-Medium', 'pr__Mellow']]

In [14]:
# Rename product strength to more user friendly names

df_strength.columns=['STRONG', 'MEDIUM-FULL', 'MODERATE', 'MELLOW-MOD.', 'MILD']

In [15]:
df_strength.head()

,STRONG,MEDIUM-FULL,MODERATE,MELLOW-MOD.,MILD
prod_name,,,,,
@ZUCAR,0,0,0,0,1
1876 Reserve,0,0,0,0,1
1876 Reserve Maduro,0,0,0,1,0
3x3 Tubos by Davidoff,0,0,0,1,0
5 Vegas Big Five,0,0,1,0,0


In [16]:
# df_main = df_main[~df_main.index.duplicated(keep='first')]
# df_main.shape

In [17]:
df_main.head()

,prod_name,rating,rating_count,desc,profile,vitolas,Brand:,Flavored,Has Tip,Origin,Pressed,Profile,Shapes,Sweet,Wrapper,Binder,Filler
0,@ZUCAR,None,None,"Vanilla and sugar... yes please!Espinosa Cigars have always prided themselves on creating cigars of superior quality that offer their fans the right cigar for every occasion, moment, or mood they may find themselves in. Fan favorites like Espinosa Crema, that many have called the perfect morning cigar, and the Laranja Reserva, a more bold flavor cigar perfect for pairing with a nightcap, are just a few of their offerings. Now, Espinosa is reaching out to expand their fan base. Their sites set on the infused cigars lovers, with their newest release aimed at giving them the chance at finding their perfect cigar with in their portfilio. Enter @ZUCAR, Espinosa's first vanilla infused, sweet-tipped cigar. Originally debuted back in 2016, with no information other than a vanilla infusion and production out of the Dominican Republic, this blend has been fully remade by the company. The new version, while still produced in the Dominican Republic, now utilizes Dominican long-filler Tobacco and an infusion of vanilla and spice to help balance the sweetness provided by the sugar sweet-tip. Sporting the same phenomenal quality in construction and components, this cigar falls right in line with all of Espinosa's previous offerings, providing an exceptional herfing experience for whatever event life may throw your way. Who knows...you may have just found your new favorite cigar!",Mellow,"[Corona, Panatela, Robusto, Toro]",Espinosa,Varies,No,Dominican Republic,False,NaN,"Corona, Panatela, Robusto, Toro",True,Ecuador Sumatra,NaN,NaN
1,1876 Reserve,4.28,559,"Even gramps would be proud.On a trip to factories in the Dominican Republic some time ago, I was offered a boatload of a really nice, smooth handmade called 1876 Reserve. I’m guessing the manufacturer either drank too much Dominican rum or his abacus was busted. Don’t get me wrong, I certainly ain’t complaining because I think under normal circumstances it would’ve cost me twice as much to buy what I bought (I better remove his name from the mailing list so he doesn’t see this later). What I do know is that this is an incredibly mellow Dominican cigar that’s well worth our ridiculous 2-fer price. In fact, the buy-in here starts under two bucks apiece! Is that too cheap? Maybe it’s me that’s getting soft in the head. Oh well, if you like easy going cigars, you’ll love this deal.",Mellow,"[Churchill, Robusto, Toro, Torpedo]",NaN,False,No,Dominican Republic,False,NaN,"Churchill, Robusto, Toro, Torpedo",False,Connecticut,Sumatra,Dominican
2,1876 Reserve Maduro,4.24,289,"A dark horse from 1876 Reserve.Maestro....drumroll please. And now, without further delay, enter 1876 Reserve Maduro. That's right, the fan-favorite 1876 Reserve now comes draped in a dark, juicy maduro wrapper leaf. The profile is still relatively tame, yet the darker, richer leaf adds a welcomed layer of complexity. And better yet, the price is exactly the same, right around two bucks apiece. I don't like taking credit when it's not due, but I definitely deserve a pat on the back for this one. Well-made Dominican lovelies stuffed with aged Dominican long-fillers and sporting a toothy maduro wrapper, all for pennies on the dollar. You can put this one at the top of the list for best damn value around.",Mellow-Medium,"[Churchill, Robusto, Toro, Torpedo]",NaN,False,No,Dominican Republic,False,NaN,"Churchill, Robusto, Toro, Torpedo",False,Maduro,NaN,Dominican
3,3x3 Tubos by Davidoff,4.53,38,"3x3 = an Unbridled Winner! There’s tons of cigars that have gone the way of the Dodo bird over the last few years… but perhaps the most sought-after blend post-disappearing act is the famed 3x3. On paper, this cigar shouldn’t work. It’s just a collection of 9 cigars, packaged in three rows of three, that each come in aluminum tubes. But the secret to this one is in the sauce! Handmade by the

In [18]:
# Set product name as the index

df_main.set_index('prod_name', inplace=True)

In [19]:
# Isolate product description into a dataframe

df_desc = pd.DataFrame(df_main['desc'])
df_desc.head()

,desc
prod_name,
@ZUCAR,"Vanilla and sugar... yes please!Espinosa Cigars have always prided themselves on creating cigars of superior quality that offer their fans the right cigar for every occasion, moment, or mood they may find themselves in. Fan favorites like Espinosa Crema, that many have called the perfect morning cigar, and the Laranja Reserva, a more bold flavor cigar perfect for pairing with a nightcap, are just a few of their offerings. Now, Espinosa is reaching out to expand their fan base. Their sites set on the infused cigars lovers, with their newest release aimed at giving them the chance at finding their perfect cigar with in their portfilio. Enter @ZUCAR, Espinosa's first vanilla infused, sweet-tipped cigar. Originally debuted back in 2016, with no information other than a vanilla infusion and production out of the Dominican Republic, this blend has been fully remade by the company. The new version, while still produced in the Dominican Republic, now utilizes Dominican long-filler Tobacco and an infusion of vanilla and spice to help balance the sweetness provided by the sugar sweet-tip. Sporting the same phenomenal quality in construction and components, this cigar falls right in line with all of Espinosa's previous offerings, providing an exceptional herfing experience for whatever event life may throw your way. Who knows...you may have just found your new favorite cigar!"
1876 Reserve,"Even gramps would be proud.On a trip to factories in the Dominican Republic some time ago, I was offered a boatload of a really nice, smooth handmade called 1876 Reserve. I’m guessing the manufacturer either drank too much Dominican rum or his abacus was busted. Don’t get me wrong, I certainly ain’t complaining because I think under normal circumstances it would’ve cost me twice as much to buy what I bought (I better remove his name from the mailing list so he doesn’t see this later). What I do know is that this is an incredibly mellow Dominican cigar that’s well worth our ridiculous 2-fer price. In fact, the buy-in here starts under two bucks apiece! Is that too cheap? Maybe it’s me that’s getting soft in the head. Oh well, if you like easy going cigars, you’ll love this deal."
1876 Reserve Maduro,"A dark horse from 1876 Reserve.Maestro....drumroll please. And now, without further delay, enter 1876 Reserve Maduro. That's right, the fan-favorite 1876 Reserve now comes draped in a dark, juicy maduro wrapper leaf. The profile is still relatively tame, yet the darker, richer leaf adds a welcomed layer of complexity. And better yet, the price is exactly the same, right around two bucks apiece. I don't like taking credit when it's not due, but I definitely deserve a pat on the back for this one. Well-made Dominican lovelies stuffed with aged Dominican long-fillers and sporting a toothy maduro wrapper, all for pennies on the dollar. You can put this one at the top of the list for best damn value around."
3x3 Tubos by Davidoff,"3x3 = an Unbridled Winner! There’s tons of cigars that have gone the way of the Dodo bird over the last few years… but perhaps the most sought-after blend post-disappearing act is the famed 3x3. On paper, this cigar shouldn’t work. It’s just a collection of 9 cigars, packaged in three rows of three, that each come in aluminum tubes. But the secret to this one is in the sauce! Handmade by the folks behind the famed Davidoff brand, 3x3 tubos are produced in the Dominican Republic and feature a silky-smooth Ecuadorian Connecticut wrapper leaf. Underneath, you’ll find Dominican binder and long-fillers, delivering a toasty, earthy, slightly spicy profile with a smooth finish you’ll certainly love."
5 Vegas Big Five,"High five! 5 Vegas's first new release in years is big. No, really. These suckers are HUGE. With each size weighing in at a hefty 60-ring gauge, these flavorful heavyweights are the all-you-can-eat buffet of the cigar world. And they're bulked up to contain that massive amount of flavor you'd expect fro

In [20]:
# Create columns of terms that describe product characteristics

df_desc['puro'] = df_desc['desc'].str.contains('puro').astype(int)
df_desc['sweet'] = df_desc['desc'].str.contains('sweet').astype(int)
df_desc['leather'] = df_desc['desc'].str.contains('leather').astype(int)
df_desc['chocolate'] = df_desc['desc'].str.contains('chocolate').astype(int)
df_desc['coffee'] = df_desc['desc'].str.contains('coffee').astype(int)
df_desc['earth'] = df_desc['desc'].str.contains('earth').astype(int)
df_desc['cedar'] = df_desc['desc'].str.contains('cedar').astype(int)
df_desc['espresso'] = df_desc['desc'].str.contains('espresso').astype(int)
df_desc['vanilla'] = df_desc['desc'].str.contains('vanilla').astype(int)
df_desc['cream'] = df_desc['desc'].str.contains('cream').astype(int)
df_desc['cocoa'] = df_desc['desc'].str.contains('cocoa').astype(int)
df_desc['caramel'] = df_desc['desc'].str.contains('caramel').astype(int)
df_desc['bread'] = df_desc['desc'].str.contains('bread').astype(int)
df_desc['pepper'] = df_desc['desc'].str.contains('pepper').astype(int)
df_desc['wood'] = df_desc['desc'].str.contains('wood').astype(int)
df_desc['nut'] = df_desc['desc'].str.contains('nuts').astype(int)
df_desc['spice'] = df_desc['desc'].str.contains('spic').astype(int)
df_desc['fruit'] = df_desc['desc'].str.contains('fruit').astype(int)
df_desc['grass'] = df_desc['desc'].str.contains('grass').astype(int)
df_desc['hay'] = df_desc['desc'].str.contains('hay').astype(int)
df_desc['honey'] = df_desc['desc'].str.contains('honey').astype(int)
df_desc['cherry'] = df_desc['desc'].str.contains('cherry').astype(int)
df_desc['cinnamon'] = df_desc['desc'].str.contains('cinnamon').astype(int)
df_desc['almond'] = df_desc['desc'].str.contains('almond').astype(int)
df_desc['oak'] = df_desc['desc'].str.contains('oak').astype(int)
df_desc['nutmeg'] = df_desc['desc'].str.contains('nutmeg').astype(int)
df_desc['ginger'] = df_desc['desc'].str.contains('ginger').astype(int)
df_desc['tannin'] = df_desc['desc'].str.contains('tanni').astype(int)
df_desc['toast'] = df_desc['desc'].str.contains('toast').astype(int)

In [21]:
df_desc2 = df_desc.drop(['desc'], axis=1)

In [22]:
df_strength.shape

(1419, 5)

In [23]:
df_desc2.head()

,puro,sweet,leather,chocolate,coffee,earth,cedar,espresso,vanilla,cream,cocoa,caramel,bread,pepper,wood,nut,spice,fruit,grass,hay,honey,cherry,cinnamon,almond,oak,nutmeg,ginger,tannin,toast
prod_name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
@ZUCAR,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1876 Reserve,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1876 Reserve Maduro,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3x3 Tubos by Davidoff,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
5 Vegas Big Five,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [24]:
# Concatenate product attributes with flavor characteristics

df_final_v_4 = pd.concat([df_final_v_3, df_desc2], axis=1)

In [25]:
# Remove duplicates

df_final_v_4 = df_final_v_4[~df_final_v_4.index.duplicated(keep='first')]
df_final_v_4.shape

(1419, 112)

In [26]:
df_final_v_4.head()

,f_african,f_brazilian,f_colombian,f_connecticut,f_costa_rican,f_dominican,f_ecuadorian,f_honduran,f_indonesian,f_italian,f_jamaican,f_kentucky_fire_cured,f_mexican,f_nicaraguan,f_panamanian,f_pennsylvania,f_peruvian,f_sungrown,f_united_states,f_varied,w_2000,w_98,w_brazilian,w_cameroon,w_candela,w_connecticut,w_connecticut_broadleaf,w_corojo,w_costa_rican,w_criollo,w_dominican,w_ecuador,w_habana,w_habano,w_honduran,w_htl,w_indonesian,w_jalapa,w_kentucky_fire_cured,w_ligero,w_maduro,w_nicaraguan,w_oscuro,w_pennsylvania_broadleaf,w_rosado,w_san_andres,w_sumatra,w_sungrown,w_varied,pr__Full,pr__Medium,pr__Medium-Full,pr__Mellow,pr__Mellow-Medium,Fl__True,Fl__Varies,Fl__nan,Tip__Varies,Tip__Yes,O__Bahamas,O__Costa Rica,O__Dominican Republic,"O__Dominican Republic, Honduras","O__Dominican Republic, Nicaragua","O__Ecuador, Europe",O__Europe,"O__Europe, Honduras",O__Honduras,"O__Honduras, Nicaragua",O__Indonesia,O__Ireland,O__Italy,O__Nicaragua,"O__Nicaragua, United States",O__Philippines,O__Sri Lanka,O__United States,Pr__Varies,Pr__Yes,Sw__Varies,Sw__Yes,rating,rating_count,puro,sweet,leather,chocolate,coffee,earth,cedar,espresso,vanilla,cream,cocoa,caramel,bread,pepper,wood,nut,spice,fruit,grass,hay,honey,cherry,cinnamon,almond,oak,nutmeg,ginger,tannin,toast
prod_name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
@ZUCAR,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.000,0.000000,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1876 Reserve,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.856,0.263059,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1876 Reserve Maduro,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.848,0.136000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3x3 Tubos by Davidoff,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.906,0.017882,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
5 Vegas Big Five,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.872,0.040941,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [27]:
# Instatiate a k-nearest neighbors algorithm with a weighted Minkowski metric. The weights are used to give less 
# weight to flavor characteristics and more weight to sweet and flavored attributes.

knn_search = NearestNeighbors(metric='wminkowski', p=2, metric_params={'w': 
    [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]})

In [28]:
knn_search.fit(df_final_v_4)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='wminkowski',
                 metric_params={'w': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
                                      1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
                                      1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
                                      1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...]},
                 n_jobs=None, n_neighbors=5, p=2, radius=1.0)

In [29]:
# Widget to look up values of products, store it's index value and display it's name. 

import autoFill as af 
from ipywidgets import HTML, HBox

def open(value):
    show.value=value.new
    
options = list(df_final_v_4.index)
autofill = af.autoFill(options,callback=open)

show = HTML('Result will be displayed here!')

display(HBox([autofill,show]))


In [32]:
# Function that returns the top 10 nearest neigbors to the product selected in the cell above

def get_key(val): 
    for key, value in dict(enumerate(options)).items(): 
         if val == value: 
            return key 
  

query_index = get_key(show.value)
distances, indices = knn_search.kneighbors(df_final_v_4.iloc[query_index,:].values.reshape(1,-1), n_neighbors=11)

for i in range(0,len(distances.flatten())):

    if i == 0:
        print('Top Cigar Recommendations for: {}'.format(df_final_v_4.index[query_index]))
    else:
        print('{}: {} with a Distance Score of: {}'.format(i, df_final_v_4.index[indices.flatten()[i]],round(distances.flatten()[i],4)))

Top Cigar Recommendations for: 601 La Bomba
1: Diesel Heart of Darkness with a Distance Score of: 1.0002
2: New Cuba Fuerte with a Distance Score of: 1.1184
3: Crowned Heads La Imperiosa with a Distance Score of: 1.2248
4: Nub Cain FF by Oliva with a Distance Score of: 1.3229
5: Alec Bradley Black Market Esteli with a Distance Score of: 1.3229
6: Hoyo La Amistad Gold with a Distance Score of: 1.323
7: Southern Draw Kudzu LUSTRUM with a Distance Score of: 1.323
8: Nica Libre Silver 25th Anniversary with a Distance Score of: 1.323
9: Rocky Patel The Edge Habano with a Distance Score of: 1.3231
10: Tatuaje Fausto with a Distance Score of: 1.3235


In [33]:
# Remove duplicates from product keywords dataframes

df_desc2= df_desc2[~df_desc2.index.duplicated(keep='first')]
df_desc2.shape

(1419, 29)

In [34]:
# Concatenate df_strength and df_desc2 to use in web application

df_desc_str = pd.concat([df_strength, df_desc2], axis=1)

In [35]:
df_desc_str.head()

,STRONG,MEDIUM-FULL,MODERATE,MELLOW-MOD.,MILD,puro,sweet,leather,chocolate,coffee,earth,cedar,espresso,vanilla,cream,cocoa,caramel,bread,pepper,wood,nut,spice,fruit,grass,hay,honey,cherry,cinnamon,almond,oak,nutmeg,ginger,tannin,toast
prod_name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
@ZUCAR,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1876 Reserve,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1876 Reserve Maduro,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3x3 Tubos by Davidoff,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
5 Vegas Big Five,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [36]:
# Create a column called 'New' that combines all the keywords that describe the product

df_desc_str['New']=df_desc_str.astype(int).dot(df_desc_str.columns+', ').str[:-1]
df_desc_str.head()

,STRONG,MEDIUM-FULL,MODERATE,MELLOW-MOD.,MILD,puro,sweet,leather,chocolate,coffee,earth,cedar,espresso,vanilla,cream,cocoa,caramel,bread,pepper,wood,nut,spice,fruit,grass,hay,honey,cherry,cinnamon,almond,oak,nutmeg,ginger,tannin,toast,New
prod_name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
@ZUCAR,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,"MILD, sweet, vanilla, spice,"
1876 Reserve,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"MILD,"
1876 Reserve Maduro,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"MELLOW-MOD.,"
3x3 Tubos by Davidoff,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,"MELLOW-MOD., earth, spice, toast,"
5 Vegas Big Five,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,"MODERATE, earth, cedar, spice,"
